In [2]:
from clique_benchmark import run_benchmark_sample, clique_oracle_compiler_classical_function, Trials, CompileType, backend
from graph_database import Graphs, Graph
from qiskit_ibm_runtime import Batch

# XAG oracle
from clique_verifier_xag.py import oracle_from_graph

graph_db = Graphs()

In [4]:
# run benchmark
for n in range(3,21):
    for p in range(1, 100, 2):
        with Batch(backend) as batch:
            for graph in graph_db.get(n=n, p=p):
                target_clique_size = max(n // 2, 2)  # search for big clique for now
                if graph.clique_counts[target_clique_size] == 0:
                    continue

                clique_oracle = oracle_from_graph(graph, target_clique_size)
                optimal_grover_iterations = graph.optimal_grover_iterations(target_clique_size)

                for grover_iterations in range(1, optimal_grover_iterations + 1):
                    run_benchmark_sample(
                        graph=graph,
                        compile_type=CompileType.XAG,
                        clique_oracle=clique_oracle,
                        clique_size=target_clique_size,
                        grover_iterations=grover_iterations,
                        shots=10**3,
                        include_existing_trials=True
                    )
            
        
    


Running benchmark sample for 001
Looking for a clique with 2 vertices using 1 iterations


base_primitive._get_mode_service_backend:WARNING:2025-04-15 16:49:33,706: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
base_primitive._get_mode_service_backend:WARNING:2025-04-15 16:49:36,037: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


Running benchmark sample for 001
Looking for a clique with 2 vertices using 2 iterations


base_primitive._get_mode_service_backend:WARNING:2025-04-15 16:49:37,760: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


Running benchmark sample for 100
Looking for a clique with 2 vertices using 1 iterations


...

base_primitive._get_mode_service_backend:WARNING:2025-04-15 17:01:52,998: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


Running benchmark sample for 0010000010
Running benchmark sample for 0010000010
Running benchmark sample for 0010000010
Running benchmark sample for 1001010100
Looking for a clique with 2 vertices using 1 iterations


base_primitive._get_mode_service_backend:WARNING:2025-04-15 17:01:55,188: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
base_primitive._get_mode_service_backend:WARNING:2025-04-15 17:01:57,109: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


Running benchmark sample for 1001010100
Looking for a clique with 2 vertices using 2 iterations


KeyboardInterrupt: 